## Code Along :: Feature Selection and Logistic regression

## About the Dataset

The "spam" concept is diverse: advertisements for products/web sites, make money fast schemes, chain letters, pornography... 

Our collection of spam e-mails came from our postmaster and individuals who had filed spam. Our collection of non-spam e-mails came from filed work and personal e-mails, and hence the word 'george' and the area code '650' are indicators of non-spam. These are useful when constructing a personalized spam filter. One would either have to blind such non-spam indicators or get a very wide collection of non-spam to generate a general purpose spam filter. 

-  Number of Instances: 4601 (1813 Spam = 39.4%)
-  Number of Attributes: 58 (57 continuous, 1 nominal class label)

 -  Attribute Information:

    -  The last column of 'spambase.data' denotes whether the e-mail was 
       considered spam (1) or not (0)
    
    - 48 attributes are continuous real [0,100] numbers of type `word freq WORD` i.e. percentage of words in the e-mail that         match WORD

    - 6 attributes are continuous real [0,100] numbers of type `char freq CHAR` i.e. percentage of characters in the e-mail           that match CHAR
    
    - 1 attribute is continuous real [1,...] numbers of type `capital run length average` i.e. average length of uninterrupted       sequences of capital letters

    - 1 attribute is continuous integer [1,...] numbers of type `capital run length longest` i.e. length of longest                   uninterrupted sequence of capital letters

    - 1 attribute is continuous integer [1,...] numbers of type `capital run length total` i.e. sum of length of uninterrupted       sequences of capital letters in the email

    - 1 attribute is nominal {0,1} class  of type spam i.e  denotes whether the e-mail was considered spam (1) or not (0),  

- Missing Attribute Values: None

- Class Distribution:
	Spam	  1813  (39.4%)
	Non-Spam  2788  (60.6%)



You can read more about dataset [here](https://archive.ics.uci.edu/ml/datasets/spambase)


In [43]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import warnings
warnings.filterwarnings("ignore")

## Loading the dataset

In [44]:
#Loading the Spam data for the mini challenge
#Target variable is the 57 column i.e spam, non-spam classes 
df = pd.read_csv('spambase.data.csv',header=None)
df.head()

0     1     2    3     4     5     6     7     8     9     10    11  \
0  0.00  0.64  0.64  0.0  0.32  0.00  0.00  0.00  0.00  0.00  0.00  0.64   
1  0.21  0.28  0.50  0.0  0.14  0.28  0.21  0.07  0.00  0.94  0.21  0.79   
2  0.06  0.00  0.71  0.0  1.23  0.19  0.19  0.12  0.64  0.25  0.38  0.45   
3  0.00  0.00  0.00  0.0  0.63  0.00  0.31  0.63  0.31  0.63  0.31  0.31   
4  0.00  0.00  0.00  0.0  0.63  0.00  0.31  0.63  0.31  0.63  0.31  0.31   

     12    13    14    15    16    17    18    19    20   21    22    23   24  \
0  0.00  0.00  0.00  0.32  0.00  1.29  1.93  0.00  0.96  0.0  0.00  0.00  0.0   
1  0.65  0.21  0.14  0.14  0.07  0.28  3.47  0.00  1.59  0.0  0.43  0.43  0.0   
2  0.12  0.00  1.75  0.06  0.06  1.03  1.36  0.32  0.51  0.0  1.16  0.06  0.0   
3  0.31  0.00  0.00  0.31  0.00  0.00  3.18  0.00  0.31  0.0  0.00  0.00  0.0   
4  0.31  0.00  0.00  0.31  0.00  0.00  3.18  0.00  0.31  0.0  0.00  0.00  0.0   

    25   26   27   28   29   30   31   32   33   34   35    36   37   38  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.07  0.0  0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.0  0.0   

     39   40   41    42   43    44    45   46   47    48     49   50     51  \
0  0.00  0.0  0.0  0.00  0.0  0.00  0.00  0.0  0.0  0.00  0.000  0.0  0.778   
1  0.00  0.0  0.0  0.00  0.0  0.00  0.00  0.0  0.0  0.00  0.132  0.0  0.372   
2  0.06  0.0  0.0  0.12  0.0  0.06  0.06  0.0  0.0  0.01  0.143  0.0  0.276   
3  0.00  0.0  0.0  0.00  0.0  0.00  0.00  0.0  0.0  0.00  0.137  0.0  0.137   
4  0.00  0.0  0.0  0.00  0.0  0.00  0.00  0.0  0.0  0.00  0.135  0.0  0.135   

      52     53     54   55    56  57  
0  0.000  0.000  3.756   61   278   1  
1  0.180  0.048  5.114  101  1028   1  
2  0.184  0.010  9.821  485  2259   1  
3  0.000  0.000  3.537   40   191   1  
4  0.000  0.000  3.537   40   191   1

### 1. Get an overview of your data by using info() and describe() functions of pandas.


In [45]:
# Overview of the data
pd.options.display.max_columns = 999
#print(df.info())
df.describe()

0            1            2            3            4   \
count  4601.000000  4601.000000  4601.000000  4601.000000  4601.000000   
mean      0.104553     0.213015     0.280656     0.065425     0.312223   
std       0.305358     1.290575     0.504143     1.395151     0.672513   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       0.000000     0.000000     0.000000     0.000000     0.000000   
75%       0.000000     0.000000     0.420000     0.000000     0.380000   
max       4.540000    14.280000     5.100000    42.810000    10.000000   

                5            6            7            8            9   \
count  4601.000000  4601.000000  4601.000000  4601.000000  4601.000000   
mean      0.095901     0.114208     0.105295     0.090067     0.239413   
std       0.273824     0.391441     0.401071     0.278616     0.644755   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       0.000000     0.000000     0.000000     0.000000     0.000000   
75%       0.000000     0.000000     0.000000     0.000000     0.160000   
max       5.880000     7.270000    11.110000     5.260000    18.180000   

                10           11           12           13           14  \
count  4601.000000  4601.000000  4601.000000  4601.000000  4601.000000   
mean      0.059824     0.541702     0.093930     0.058626     0.049205   
std       0.201545     0.861698     0.301036     0.335184     0.258843   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       0.000000     0.100000     0.000000     0.000000     0.000000   
75%       0.000000     0.800000     0.000000     0.000000     0.000000   
max       2.610000     9.670000     5.550000    10.000000     4.410000   

                15           16           17           18           19  \
count  4601.000000  4601.000000  4601.000000  4601.000000  4601.000000   
mean      0.248848     0.142586     0.184745     1.662100     0.085577   
std       0.825792     0.444055     0.531122     1.775481     0.509767   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       0.000000     0.000000     0.000000     1.310000     0.000000   
75%       0.100000     0.000000     0.000000     2.640000     0.000000   
max      20.000000     7.140000     9.090000    18.750000    18.180000   

                20           21           22           23           24  \
count  4601.000000  4601.000000  4601.000000  4601.000000  4601.000000   
mean      0.809761     0.121202     0.101645     0.094269     0.549504   
std       1.200810     1.025756     0.350286     0.442636     1.671349   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       0.220000     0.000000     0.000000     0.000000     0.000000   
75%       1.270000     0.000000     0.000000     0.000000     0.000000   
max      11.110000    17.100000     5.450000    12.500000    20.830000   

                25           26           27           28           29  \
count  4601.000000  4601.000000  4601.000000  4601.000000  4601.000000   
mean      0.265384     0.767305     0.124845     0.098915     0.102852   
std       0.886955     3.367292     0.538576     0.593327     0.456682   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       0.000000     0.000000     0.000000     0.000000     0.000000   
75%       0.000000     0.000000     0.000000     0.000000     0.000000   
max      16.660000    33.330000     9.090000    14.280000     5.880000   

              

### 2. Split the data into train and test set and fit the base logistic regression model on train set.

In [48]:
#Dividing the dataset set in train and test set and apply base logistic model
from sklearn.linear_model import LogisticRegression
print(df.shape)
X = df.iloc[:,:57]
print(X.shape)
y = df.iloc[:,-1]
print(y.shape)
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size = 0.2 , random_state = 5)
model = LogisticRegression()
model.fit(X_train,y_train)

(4601, 58)
(4601, 57)
(4601,)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

### 3. Find out the accuracy , print out the Classification report and Confusion Matrix.

In [49]:
# Calculate accuracy , print out the Classification report and Confusion Matrix.

print('Accuracy',model.score(X_test,y_test))

y_pred = model.predict(X_test)
print('Accuracy Score',accuracy_score(y_test,y_pred))
print('Classification Report \n',classification_report(y_test,y_pred))
print('Confusion Matrix \n', confusion_matrix(y_test,y_pred))

Accuracy 0.9272529858849077
Accuracy Score 0.9272529858849077
Classification Report 
               precision    recall  f1-score   support

           0       0.93      0.95      0.94       575
           1       0.92      0.88      0.90       346

   micro avg       0.93      0.93      0.93       921
   macro avg       0.93      0.92      0.92       921
weighted avg       0.93      0.93      0.93       921

Confusion Matrix 
 [[548  27]
 [ 40 306]]


### 4. Copy dataset df into df1 variable and apply correlation on df1

In [50]:
# Copy df in new variable df1
df1 = df

In [51]:
df_corr = df1.corr()

### 5. As we have learned  one of the assumptions of Logistic Regression model is that the independent features should not be correlated to each other(i.e Multicollinearity), So we have to find the features that have a correlation higher that 0.75 and remove the same so that the assumption for logistic regression model is satisfied. 

In [52]:
# Remove Correlated features above 0.75 and then apply logistic model
corr_matrix = df1.drop([57],axis = 1).corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.
to_drop = [column for column in upper.columns if any(upper[column] > 0.75)]
print("Columns to be dropped: ")
print(to_drop)

Columns to be dropped: 
[33, 39]


In [53]:
print("Columns to be dropped: ")
df1.drop(to_drop,axis=1,inplace=True)

Columns to be dropped: 


### 6. Split the  new subset of the  data acquired by feature selection into train and test set and fit the logistic regression model on train set.

In [54]:
# Split the new subset of data and fit the logistic model on training data
X_train,X_test,y_train,y_test= train_test_split(df1.loc[:,:56],df1[57],test_size = 0.2 , random_state = 5)
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

### 7. Find out the accuracy , print out the Classification report and Confusion Matrix.

In [55]:
# Calculate accuracy , print out the Classification report and Confusion Matrix for new data
print('Accuracy',model.score(X_test,y_test))

y_pred = model.predict(X_test)
print('Accuracy Score',accuracy_score(y_test,y_pred))
print('Classification Report \n',classification_report(y_test,y_pred))
print('Confusion Matrix \n', confusion_matrix(y_test,y_pred))

Accuracy 0.9261672095548317
Accuracy Score 0.9261672095548317
Classification Report 
               precision    recall  f1-score   support

           0       0.93      0.95      0.94       575
           1       0.92      0.88      0.90       346

   micro avg       0.93      0.93      0.93       921
   macro avg       0.92      0.92      0.92       921
weighted avg       0.93      0.93      0.93       921

Confusion Matrix 
 [[547  28]
 [ 40 306]]


### 8. After keeping highly correlated features, there is not much change in the score. Lets apply another feature selection technique(Chi Squared test) to see whether we can increase our score. Find the optimum number of features using Chi Square and fit the logistic model on train data.

In [58]:
# Apply Chi Square and fit the logistic model on train data use df dataset
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
test = SelectKBest(score_func = chi2,k= 25) 
X_train,X_test,y_train,y_test  = train_test_split(df1.loc[:,:56],df1[57],test_size =0.2,random_state = 5)
X_train = test.fit_transform(X_train,y_train)
X_test = test.transform(X_test)

model = LogisticRegression()
model.fit(X_train,y_train)
score = model.score(X_test,y_test)
score

0.9218241042345277

### 9. Find out the accuracy , print out the Confusion Matrix.

In [11]:
# Calculate accuracy , print out the Confusion Matrix 


### 10. Using chi squared test there is no change in the score and the optimum features that we got is 55. Now lets see if we can increase our score using another feature selection technique called Anova.Find the optimum number of features using Anova and fit the logistic model on train data.

In [12]:
# Apply Anova and fit the logistic model on train data use df dataset


### 11. Find out the accuracy , print out the Confusion Matrix.

In [13]:
# Calculate accuracy , print out the Confusion Matrix 


### 12. Unfortunately Anova also couldn't give us a better score . Let's finally attempt PCA on train data and find if it helps in  giving a better model by reducing the features.

In [14]:
# Apply PCA and fit the logistic model on train data use df dataset


### 13. Find out the accuracy , print out the Confusion Matrix.   

In [15]:
# Calculate accuracy , print out the Confusion Matrix 


### 14. You can also compare your predicted values and observed values by printing out values of logistic.predict(X_test[]) and  y_test[].values

In [16]:
# Compare observed value and Predicted value
